In [10]:
import pandas as pd
import gzip
from collections import *
import geopandas
from shapely.geometry import Point
from geopandas.tools import sjoin

In [ ]:
uid_to_loc = {}


with gzip.open('/shared/1/projects/location-inference/data/locations.2012-2019.from-merged.all.4-iter.tsv_iter_3.tsv.gz') as f:
    loc_data = pd.read_csv(f, sep='\t', names = ["user_id", "latitude", "longitude", "unnamed"], index_col=False)

In [ ]:
loc_data.head()

In [ ]:
#this is switched with respect to the labels, should fix above

gdf = geopandas.GeoDataFrame(
    loc_data, geometry=geopandas.points_from_xy(loc_data.longitude, loc_data.latitude))
print(gdf.head())

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
print(world.head())

In [ ]:
import rtree
gdf.crs = {'init' :'epsg:4326'}
world = world[["name", "geometry"]]
loc_data = geopandas.sjoin(gdf, world, how="left", op="within")
print(loc_data)

In [1]:
relevant_countries = [('India', 'IN'), ('Jamaica', 'JM'), ('Nigeria', 'NG'),('Philippines', 'PH'), ('Bahamas', 'BS'), ('Mexico', 'MX'), ('Ghana', 'GH'), ('Sri Lanka', 'LK'), ('Pakistan', 'PK')]
print(relevant_countries)

[('India', 'IN'), ('Jamaica', 'JM'), ('Nigeria', 'NG'), ('Senegal', 'SN'), ('Singapore', 'SG'), ('Philippines', 'PH'), ('Bahamas', 'BS'), ('Mexico', 'MX'), ('Ghana', 'GH'), ('Sri Lanka', 'LK'), ('Pakistan', 'PK')]


In [13]:
for country in relevant_countries:
    relevant_users = loc_data.loc[(loc_data.name==country[0])].copy()
    relevant_users.rename(index=str, columns={"name": "country"}, inplace=True)
    relevant_users.drop(columns=['latitude', 'longitude', 'geometry', 'index_right', 'unnamed'], inplace=True)
    relevant_users.set_index('user_id')
    relevant_users.to_csv(r'/home/sayghosh/hate-speech-examination/data/user_lists/' + country[1] + '.tsv', sep='\t', encoding='utf-8')